<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Limpieza-de-los-Datos" data-toc-modified-id="Limpieza-de-los-Datos-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Limpieza de los Datos</a></span><ul class="toc-item"><li><span><a href="#ColumnTransformer" data-toc-modified-id="ColumnTransformer-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>ColumnTransformer</a></span></li><li><span><a href="#Ejemplo-1:" data-toc-modified-id="Ejemplo-1:-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Ejemplo 1:</a></span></li><li><span><a href="#Ejercicio-5:" data-toc-modified-id="Ejercicio-5:-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Ejercicio 5:</a></span></li><li><span><a href="#Resolución-Ejercicio-5:" data-toc-modified-id="Resolución-Ejercicio-5:-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Resolución Ejercicio 5:</a></span></li><li><span><a href="#ColumnTransformer-con-selección-de-Columnas-según-su-tipo" data-toc-modified-id="ColumnTransformer-con-selección-de-Columnas-según-su-tipo-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>ColumnTransformer con selección de Columnas según su tipo</a></span><ul class="toc-item"><li><span><a href="#Ejercicio-6" data-toc-modified-id="Ejercicio-6-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Ejercicio 6</a></span></li><li><span><a href="#Resolución-Ejercicio-6" data-toc-modified-id="Resolución-Ejercicio-6-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Resolución Ejercicio 6</a></span></li></ul></li><li><span><a href="#RESUMEN" data-toc-modified-id="RESUMEN-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>RESUMEN</a></span></li></ul></li></ul></div>

![IES21](img/logo_ies.png)

# Situación Profesional 4: Preparación de los Datos

In [1]:
import numpy as np
import pandas as pd

## Limpieza de los Datos

Con anterioridad hemos visto cómo utilizar el SimpleImputer de Scikit-Learn para imputar valores a los nulos que teníamos en nuestros datasets.   
Es muy habitual que, como nos ocurrió en el último problema, algunas columnas deban recibir cierto tratamiento de imputación y otras columnas otro, ya sea por el tipo de dato de cada una (numérico / categórico) o por necesidad de aplicar la media o la mediana a distintas columnas.   

Seguramente observó que la tarea es un tanto tediosa, más que nada por tener que armar dataframes separados para cada caso, y luego tener que juntarlos nuevamente ... por suerte sklearn nos puede proveer de algunos atajos ... por ejemplo para efectuar **transformaciones sobre las columnas**, como han sido nuestras imputaciones, podemos ahorrarnos algo de tiempo utilizando a **ColumnTransformer** ...

### ColumnTransformer

Como mencionábamos, ColumnTransformer es una clase que nos ahorrará algo del trabajo pesado al trabajar con columnas en la Preparación de los Datos (no sólo para imputar, ya veremos), y nos permitirá definir una secuencia de transformaciones (como las imputaciones) y luego ejecutarlas todas "juntas".  

ColumnTransformer esencialmente ejecuta una **lista de transformaciones** que queremos aplicar a un DataFrame de Pandas o un array de Numpy. La documentación oficial es la siguiente:  

https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

La sintaxis es la siguiente:  

sklearn.compose.ColumnTransformer(transformers, *, remainder='drop', sparse_threshold=0.3, n_jobs=None, transformer_weights=None, verbose=False)  

De la cual lo más importante es:  

- **transformers**: es la lista de transformaciones que queremos efectuar sobre las columnas (por ejeplo SimpleImputer)  

- **remainder**: puede ser que algunas columnas no se vean afectadas por las distintas transformaciones, si dejamos el valor en **'drop'** **no** las incluirá en el resultado, si establecemos el calor como **'passthrough'** entonces sí las dejará en el resultado. **Generalmente utilizo este último valor**. 

Se importa así:  

from sklearn.compose import ColumnTransformer



Cada uno de los transformers (transformaciones) de la lista es una tupla de 3 elementos:  

t1 = (nombre, estimador a usar p.ej SimpleImputer , columnas afectadas (podemos indicar índice o nombre!) )  


Si bien podríamos escribir todo en la sintaxis de ColumnTransformer, es más legible definir las trasnformaciones y luego pasarlas, entonces podríamos tener varias transformaciones: 

t1=(nombre , acción  , columnas)  
t2=(nombre , acción  , columnas)  
t3=(nombre , acción  , columnas) 
  
y luego definir

ColumnsTransformer(transformers=[t1,t2,t3], reminder='passthrough')  


Veamos un ejemplo y verá que fácil es de usar:

### Ejemplo 1:

Resolvamos el último problema de anterior, con el DataFrame dfD al que le hemos agregado una columna más, x4

In [2]:
dfD=pd.DataFrame([[3,2,'Si',8],[np.nan,6,'No',8],[5,8,'Si',8],[0,np.nan,'Si',8],[np.nan,4,'No',8],[6,6,np.nan,8],[4,12,'No',8],[2,9,np.nan,8]])
dfD.columns=['x1','x2','x3','x4']
dfD

,x1,x2,x3,x4
0,3.0,2.0,Si,8
1,NaN,6.0,No,8
2,5.0,8.0,Si,8
3,0.0,NaN,Si,8
4,NaN,4.0,No,8
5,6.0,6.0,NaN,8
6,4.0,12.0,No,8
7,2.0,9.0,NaN,8


Queremos imputar el promedio en las columnas numéricas y la moda en las columnas categóricas (esto es un problema bien típico de ML).  Queremos el resultado en un único DataFrame que se llame dfD_Transformado.

In [3]:
from sklearn.impute import SimpleImputer

In [4]:
from sklearn.compose import ColumnTransformer

y como siempre decimos ... son sólo 3 líneas de código!

In [5]:
# Definimos las transformaciones (transformers) que queremos utilizar:

t1=('imputador_num', SimpleImputer(strategy='mean'), ['x1', 'x2'])
t2=('imputador_cat', SimpleImputer(strategy='most_frequent'), [2])

# Creamos nuestro transformador de columnas y le pasamos todos los trasnformers que deseamos usar

transformador_columnas = ColumnTransformer(transformers=[t1,t2],remainder='passthrough')

# Lo entrenamos con el dataFrame:
transformador_columnas.fit(dfD);

Observe que:  

- en t1 pasamos las columnas por nombre y en el t2 por índice, puede usarse de una u otra forma según conveniencia!  
- como definimos remainder='passthrough', si algunas columnas no se hubieran trasnformado como la columna x4, igual las pasaría en el resultado. 

In [6]:
# Finalmente lo aplicamos con transform al DataFrame original!

D_Transformado = transformador_columnas.transform(dfD)
D_Transformado

array([[3.0, 2.0, 'Si', 8],
       [3.3333333333333335, 6.0, 'No', 8],
       [5.0, 8.0, 'Si', 8],
       [0.0, 6.714285714285714, 'Si', 8],
       [3.3333333333333335, 4.0, 'No', 8],
       [6.0, 6.0, 'No', 8],
       [4.0, 12.0, 'No', 8],
       [2.0, 9.0, 'No', 8]], dtype=object)

Observe que la columna x4 está dentro del resultado. 
El resultado es un array de Numpy, si necesitáramos que fuera un DataFrame de Pandas, lo hacemos:

In [7]:
dfD_Transformado=pd.DataFrame(D_Transformado, columns=dfD.columns)
dfD_Transformado

,x1,x2,x3,x4
0,3,2,Si,8
1,3.33333,6,No,8
2,5,8,Si,8
3,0,6.71429,Si,8
4,3.33333,4,No,8
5,6,6,No,8
6,4,12,No,8
7,2,9,No,8


### Ejercicio 5:

Podría repetir el ejericio 3 visto anteriormente:  

Dado el DataFrame dfC: 

- En las columnas A y C, impute a los valores -1, el promedio.
- En las columnas B y D impute a los valores -1 la mediana.

Guarde los resutados en un nuevo DataFrame denominado dfC_Transformado2

In [6]:
dfC=pd.DataFrame([[3,7,-1,0],[2,-1,5,4],[0,0,0,0],[-1,5,4,2],[2,6,8,1],[2,1,3,2]])
dfC.columns=['A','B','C','D']
dfC

,A,B,C,D
0,3,7,-1,0
1,2,-1,5,4
2,0,0,0,0
3,-1,5,4,2
4,2,6,8,1
5,2,1,3,2


### Resolución Ejercicio 5:

In [7]:
# Escriba aquí su código

# Definimos las transformaciones (transformers) que queremos utilizar:

t1=('imputador_prom', SimpleImputer(missing_values=-1,strategy='mean'), ['A', 'C'])
t2=('imputador_med', SimpleImputer(missing_values=-1,strategy='median'), ['B','D'])

# Creamos nuestro transformador de columnas y le pasamos todos los trasnformers que deseamos usar

transformador_columnas = ColumnTransformer(transformers=[t1,t2],remainder='passthrough')

# Lo entrenamos con el dataFrame:
transformador_columnas.fit(dfC)

ColumnTransformer(remainder='passthrough',
                  transformers=[('imputador_prom',
                                 SimpleImputer(missing_values=-1), ['A', 'C']),
                                ('imputador_med',
                                 SimpleImputer(missing_values=-1,
                                               strategy='median'),
                                 ['B', 'D'])])

In [9]:
# Finalmente lo aplicamos con transform al DataFrame original!

C_Transformado = transformador_columnas.transform(dfC)
#C_Transformado

dfC_Transformado2=pd.DataFrame(C_Transformado, columns=dfC.columns)
dfC_Transformado2

,A,B,C,D
0,3.0,4.0,7.0,0.0
1,2.0,5.0,5.0,4.0
2,0.0,0.0,0.0,0.0
3,1.8,4.0,5.0,2.0
4,2.0,8.0,6.0,1.0
5,2.0,3.0,1.0,2.0


### ColumnTransformer con selección de Columnas según su tipo

Muchas veces nos encontraremos con que tenemos que trabajar con una enorme cantidad de variables o columnas de distinto tipo (dtypes), algunas numéricas (dtype: int64 o float64), otras categóricas (dtype: object), otras booleanas (dtype: bool), y generalmente a cada tipo le aplicaremos la misma imputación, por ejemplo a las numéricas  el promedio, a las categóricas y a las booleanas, la moda, etc.  

En ese caso puede ser útil obtener los nombres de las columnas de cada tipo para aplicarles la transformación correspondiente sin tener que escribirlos de a uno por vez.  

Veamos cómo podemos abreviarnos el trabajo en el siguiente ejemplo: 

#### Ejercicio 6
Supongamos que contamos con el DataFrame dfE:

In [10]:
dfE=pd.DataFrame([[1.5,'Si',2,True],[4,'No',7,False],[np.nan,'Si',9,True],[3.5,np.nan,8,np.nan],[6,'Si',np.nan,True]])
dfE.columns=['x1','x2','x3','x4']
dfE

,x1,x2,x3,x4
0,1.5,Si,2.0,True
1,4.0,No,7.0,False
2,NaN,Si,9.0,True
3,3.5,NaN,8.0,NaN
4,6.0,Si,NaN,True


y queremos imputar: 
- el **promedio** a las numéricas y,
- la **moda** a las categóricas, de texto y booleanas, 
- el resultado lo queremos en el DataFrame denominado dfE_Transformado

Veamos de qué tipo es cada columna con info()

In [11]:
dfE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      4 non-null      float64
 1   x2      4 non-null      object 
 2   x3      4 non-null      float64
 3   x4      4 non-null      object 
dtypes: float64(2), object(2)
memory usage: 288.0+ bytes


- A x1 y x3 las tomó como float64, es decir numéricas, si hubieran sido números enteros las hubier considerado como int64. No hay diferencia para la imputación.  

- A x2 la tomó como object, que es el tipo correspondiente a los textos y categóricas.  

- Fíjese que a la columna x4 la tomó como object cuando los valores eran esencialmente booleanos, esto ocurrió por la presencial del nan. Generalmente no traerá problemas porque en ambos casos la imputación suele ser la moda. 


Esta vez no queremos pasar explícitamente los nombres de cada columna a cada trasnformer, sino que buscamos alguna manera de hacerlo programáticamente.  

Podemos usar a select_dtypes de Pandas para seleccionar las columnas según el tipo de dato:

In [13]:
cols_num=dfE.select_dtypes(np.number).columns
# en vez de np.number podriamos haber usado select_dtypes.(include=['int64','float64']).columns
cols_num

Index(['x1', 'x3'], dtype='object')

Como podemos ver, nos devolvió x1 y x3 como deseábamos. De forma similar: 

In [14]:
cols_cat=dfE.select_dtypes(include=['object','bool']).columns
cols_cat

Index(['x2', 'x4'], dtype='object')

Nos devolvió x2 y x4

Ahora sí, estamos listos! Se anima a terminarlo?

#### Resolución Ejercicio 6

Son sólo unas pocas líneas de código!

In [16]:
# Escriba aquí su código

# Definimos las transformaciones (transformers) que queremos utilizar:

t1=('imputador_num', SimpleImputer(strategy='mean'), cols_num)
t2=('imputador_txt', SimpleImputer(strategy='most_frequent'),cols_cat)

# Creamos nuestro transformador de columnas y le pasamos todos los trasnformers que deseamos usar

transformador_columnas = ColumnTransformer(transformers=[t1,t2],remainder='passthrough')

# Lo entrenamos con el dataFrame:
transformador_columnas.fit(dfE)

ColumnTransformer(remainder='passthrough',
                  transformers=[('imputador_num', SimpleImputer(),
                                 Index(['x1', 'x3'], dtype='object')),
                                ('imputador_txt',
                                 SimpleImputer(strategy='most_frequent'),
                                 Index(['x2', 'x4'], dtype='object'))])

Lo transformamos en un DataFrame de Pandas según lo solicitado:

In [17]:
# Finalmente lo aplicamos con transform al DataFrame original!

E_Transformado = transformador_columnas.transform(dfE)
#C_Transformado

dfE_Transformado=pd.DataFrame(E_Transformado, columns=dfE.columns)
dfE_Transformado

,x1,x2,x3,x4
0,1.5,2,Si,True
1,4,7,No,False
2,3.75,9,Si,True
3,3.5,8,Si,True
4,6,6.5,Si,True


Listo!

### RESUMEN

**Nota** Fíjese que si siempre fuéramos a imputar de la misma manera (el promedio a las numéricas, la moda al resto) y los Missing_values fueran siempre del mismo tipo, podríamos dejarnos escrito un código desde donde definimos cols_num hasta la última línea y lo único que cambió es el nombre del DataFrame original (y que en este caso lo cambiamos porque estábamos trabajando con varios en el mismo documento de Jupyter ...!!!  

Ese código podría ser algo así: 

In [15]:
# ver en el archivo resuelto 